# Iterating over connections with `iter_connections`

## Preamble
The code in this section assumes that you have already downloaded the circuit. If not, take a look at the [first notebook](./01_node_properties.ipynb) in the series.

In [1]:
import bluepysnap
import numpy as np
from time import time

circuit_path = "sonata/circuit_sonata.json"
circuit = bluepysnap.Circuit(circuit_path)

## Introduction
As mentioned before, due to the huge number of edges, we may run into memory issues. Therefore, it's highly recommended to use iterators instead of gathering all of the data at once. For this very reason, SNAP has `iter_connections`:
```python
edge_population.iter_connections(
    source,                   # the source nodes / query
    target,                   # the target nodes / query
    unique_node_ids=False,    # only use each source/target id once
    shuffle=False,            # shuffle the order of results
    return_edge_ids=False,    # return also the edge ids
    return_edge_count=False,  # return the edge count between the source-target pairs
)
# Returns a generator of tuples containing:
# (source_id, target_id)             : normally
# (source_id, target_id, edge_ids)   : if return_edge_ids=True
# (source_id, target_id, edge_count) : if return_edge_count=True
```
**NOTE:** `return_edge_ids` and `return_edge_count` are mutually exclusive options.

In a nutshell, what `iter_connections` does, is that it iterates through **all** of the existing connections (source-target pairs) from **any** of the **source nodes** to **any** of the **target nodes** and returns a generator yielding those source-target pairs.

Let's look at a few examples.

## Return value is a generator that we can iterate over
This is just to empahasize that we don't get the results of the function until we actually loop over it:

In [2]:
source_ids = [1]
target_ids = [27204]
edge_population = circuit.edges['thalamus_neurons__thalamus_neurons__chemical']

it = edge_population.iter_connections(source_ids, target_ids)
print(f"The result is not a tuple or a list but a {type(it)}")

The result is not a tuple or a list but a <class 'generator'>


Now, we could convert the result to a list using `list(it)` or `[*it]` but that kind of defeats the purpose of using generators and iterators. We'll just loop through them in the examples to not reinforce "bad habits".

## No optional flags set
This example is just to demonstrate that without `return_edge_ids`/`return_edge_count`, we're merely getting the source and target nodes ids as output:

In [3]:
for _source_id, _target_id in edge_population.iter_connections(source_ids, target_ids):
    print(_source_id, '-', _target_id)

CircuitNodeId(population='thalamus_neurons', id=1) - CircuitNodeId(population='thalamus_neurons', id=27204)


## Returning edge ids

By setting `return_edge_ids=True`, we get the ids of the edges connecting each source-target pair:

In [4]:
for _source_id, _target_id, _edge_ids in edge_population.iter_connections(source_ids, target_ids, return_edge_ids=True):
    print(_source_id, '-', _target_id)
    print(f'\n{_edge_ids}')

CircuitNodeId(population='thalamus_neurons', id=1) - CircuitNodeId(population='thalamus_neurons', id=27204)

CircuitEdgeIds([('thalamus_neurons__thalamus_neurons__chemical', 11570852),
            ('thalamus_neurons__thalamus_neurons__chemical', 11570853),
            ('thalamus_neurons__thalamus_neurons__chemical', 11570854),
            ('thalamus_neurons__thalamus_neurons__chemical', 11570855)],
           names=['population', 'edge_ids'])


## Returning the number of connecting edges

By setting `return_edge_count=True`, we get the number of edges connecting each source-target pair. Based on the previous example, we should be getting four connecting edges:

In [5]:
for _source_id, _target_id, _edge_count in edge_population.iter_connections(source_ids, target_ids, return_edge_count=True):
    print(_source_id, '-', _target_id)
    print(f'Edge count: {_edge_count}')

CircuitNodeId(population='thalamus_neurons', id=1) - CircuitNodeId(population='thalamus_neurons', id=27204)
Edge count: 4


## Randomizing the output order

We can use `shuffle=True` To randomize the order of the results. 

So let's see the non-randomized order of the connections between the first 10 nodes. For easier reading, let's only print the numeric part of the `CircuitNodeIds`:

In [6]:
it = enumerate(edge_population.iter_connections(range(10), range(10)))
for i, (_source_id, _target_id) in it:
    print(f'{i+1:2d}: source: {_source_id.id:2d} --- target: {_target_id.id:2d}')

 1: source:  2 --- target:  0
 2: source:  0 --- target:  4
 3: source:  5 --- target:  4
 4: source:  1 --- target:  5


Now by setting the `shuffle` flag in the call, we'll get the above source-target pairs in a different order:

In [7]:
np.random.seed(0) # Just to keep the results consistent in the notebook

it = enumerate(edge_population.iter_connections(range(10), range(10), shuffle=True))
for i, (_source_id, _target_id) in it:
    print(f'{i+1:2d}: source: {_source_id.id:2d} --- target: {_target_id.id:2d}')

 1: source:  5 --- target:  4
 2: source:  0 --- target:  4
 3: source:  1 --- target:  5
 4: source:  2 --- target:  0


## Using each node only once (at max.) as a source and as a target

Let's look at the connections between first 15 node ids. For easier reading, again, let's only print the numeric part of the `CircuitNodeId`s:

In [8]:
it = enumerate(edge_population.iter_connections(range(15), range(15)))
for i, (_source_id, _target_id) in it:
    print(f'{"W"+str(i+1):3s}: source: {_source_id.id:2d} --- target: {_target_id.id:2d}')

W1 : source:  2 --- target:  0
W2 : source: 10 --- target:  0
W3 : source: 14 --- target:  0
W4 : source:  0 --- target:  4
W5 : source:  5 --- target:  4
W6 : source:  1 --- target:  5
W7 : source: 13 --- target:  5
W8 : source: 11 --- target:  6
W9 : source:  8 --- target: 10
W10: source:  2 --- target: 14
W11: source: 13 --- target: 14


As we can see, we have 11 different source-target pairs. Note that the indices are prefixed with `W` (stands for Without a flag) to distinct them from the following. Let's see what happens when we set `unique_node_ids=True`:

In [9]:
it = enumerate(edge_population.iter_connections(range(15), range(15), unique_node_ids=True))
for i, (_source_id, _target_id) in it:
    print(f'{"U"+str(i+1):3s}: source: {_source_id.id:2d} --- target: {_target_id.id:2d}')

U1 : source:  2 --- target:  0
U2 : source:  0 --- target:  4
U3 : source:  1 --- target:  5
U4 : source: 11 --- target:  6
U5 : source:  8 --- target: 10
U6 : source: 13 --- target: 14


Cool, we've effectively lost 5 pairs of source-target pairs somewhere. 

So what happened here? The indices were prefixed with `U` (unique nodes only) to distinct them from the previous output. Let's go through the output and indices `W1`-`W11` of the previous example without `unique_node_ids` flag set and compare it to the output above:
* `W1`: kept (`U1`)
* `W2`,`W3`: removed (id `0` used as a **target** in `W1`)
* `W4`: kept (`U2`)
* `W5`: removed (id `4` used as a **target** in `W4`)
* `W6`: kept (`U3`)
* `W7`: removed (id `5` used as a **target** in `W6`)
* `W8`: kept (`U4`)
* `W9`: kept (`U5`)
* `W10`: removed (id `2` used as a **source** in `W1`)
* `W11`: kept (`U6`)

## _"Please tell me the above also works with queries"_
What kind of a software you think we're running here, pal? 

Obviously, `iter_connections` can also be called with any of the accepted node queries. The ids will be resolved on the fly:

In [10]:
it = edge_population.iter_connections(
    'mc2;VPL',            # node set
    {'region': 'mc2;Rt'}, # dict query
)
for i, (_source_id, _target_id) in enumerate(it):
    if i == 10: # Let's only print first 10
        print(f'{i+1:2d}: ...')
        break
    print(f'{i+1:2d}: source: {_source_id.id:2d} --- target: {_target_id.id:2d}')

 1: source: 33550 --- target: 28603
 2: source: 33743 --- target: 28603
 3: source: 33794 --- target: 28603
 4: source: 33818 --- target: 28603
 5: source: 34043 --- target: 28603
 6: source: 34773 --- target: 28603
 7: source: 34942 --- target: 28603
 8: source: 35126 --- target: 28603
 9: source: 35169 --- target: 28603
10: source: 35579 --- target: 28603
11: ...


## Performance optimizations

Now that we understand how `iter_connections` works, what can we do with it? What is the magic therein? 

Well, it's not really about _what_ it can do but _how_ it does it. As mentioned before, the whole purpose of using the iterators is to be memory efficient. Where it especially shines are the cases in which you are handling large number of nodes/edges and aren't necessarily interested in all of the data collected in the process.

Let's take a look at an example.

### CASE: Synapses between node sets

To demonstrate the magick of `iter_connections`, let's have a simple, straightforward example. We want to count the number of synapses between two node sets.

Now, we're not interested in the individual edge ids, just the number of synapses between two node sets. Perhaps we'd also like some statistics on how many of them are there on average between each of the source-target node pair, what is the deviation, etc.

Let's first define a source and a target node set and a helper function for printing the stats:

In [11]:
source_node_set = 'mc2;Rt'
target_node_set = 'mc2;VPL'

def print_statistics(pair_syns):
    print(f"There is a total of {np.sum(pair_syns)} synapses from '{source_node_set}' to '{target_node_set}'")
    print("\nSynapses between source-target node pairs:")
    print(f"- avg: {np.mean(pair_syns):.2f}")
    print(f"- std: {np.std(pair_syns):.2f}")
    print(f"- min: {np.min(pair_syns)}")
    print(f"- max: {np.max(pair_syns)}")

print(f'Number of source nodes: {len(edge_population.source.ids(source_node_set))}')
print(f'Number of target nodes: {len(edge_population.target.ids(target_node_set))}')

Number of source nodes: 4909
Number of target nodes: 8999


and now let's get the synapses and print the statistics:

In [12]:
t0 = time()

it = edge_population.iter_connections(source_node_set, target_node_set, return_edge_count=True)
pairwise_syns = np.fromiter((count for _,__,count in it), dtype=int)
print_statistics(pairwise_syns)

print(f'\nRuntime: {time()-t0:.2f} seconds')

There is a total of 3245906 synapses from 'mc2;Rt' to 'mc2;VPL'

Synapses between source-target node pairs:
- avg: 4.86
- std: 4.23
- min: 1
- max: 95

Runtime: 12.89 seconds


This is how we'd achieve the same with the a bit more memory-heavy approach:

In [13]:
t0 = time()
result = edge_population.pathway_edges(source_node_set,target_node_set, properties=['@source_node', '@target_node'])
print_statistics(result.value_counts().values)
print(f'\nRuntime: {time()-t0:.2f} seconds')

There is a total of 3245906 synapses from 'mc2;Rt' to 'mc2;VPL'

Synapses between source-target node pairs:
- avg: 4.86
- std: 4.23
- min: 1
- max: 95

Runtime: 5.23 seconds


### _"Dude, you just told me that `iter_connections` is supposed to be awesome, why is it slower?"_

Well spotted. There are cases, in which `iter_connections` is actually outperformed (runtime-wise) by the memory-heavy options. Worry not, we're merely warming up here. Let's shift gears and introduce a significantly bigger target node set:

In [14]:
target_node_set = 'VPL_TC'
print(f'Number of source nodes: {len(edge_population.source.ids(source_node_set))}')
print(f'Number of target nodes: {len(edge_population.target.ids(target_node_set))}')

Number of source nodes: 4909
Number of target nodes: 64856


We bumped up the number of target nodes by roughly one order of magnitude. The source nodes were left intact. Now, let's see what happens to the runtimes. Let's first run the `iter_connections` version:

In [15]:
t0 = time()
it = edge_population.iter_connections(source_node_set, target_node_set, return_edge_count=True)
pairwise_syns = np.fromiter((count for _,__,count in it), dtype=int)
print_statistics(pairwise_syns)
print(f'\nRuntime: {time()-t0:.2f} seconds')

There is a total of 4706551 synapses from 'mc2;Rt' to 'VPL_TC'

Synapses between source-target node pairs:
- avg: 4.63
- std: 3.96
- min: 1
- max: 95

Runtime: 102.22 seconds


That took quite some time. Let's see how the previously faster, `pathway_edges` implementation performs:

In [16]:
t0 = time()
result = edge_population.pathway_edges(source_node_set,target_node_set, properties=['@source_node', '@target_node'])
print_statistics(result.value_counts().values)
print(f'\nRuntime: {time()-t0:.2f} seconds')

There is a total of 4706551 synapses from 'mc2;Rt' to 'VPL_TC'

Synapses between source-target node pairs:
- avg: 4.63
- std: 3.96
- min: 1
- max: 95

Runtime: 130.58 seconds


There is a significant drop in performance in comparison to `iter_connections`, even though the number of edges/synapses wasn't _that_ much higher. Why the difference?

By not going into too much of technicalities, it boils down to:
* `pathway_synapses` needs to handle all the data at once
   * it needs to get all connecting edges and their source and target nodes
   * from the huge dataframe, it needs to find unique source-target pairs and count how many times they appear
      * `pandas.value_counts()` leads to creating another dataframe which consumes even more memory  
   * everything is kept in memory throughout the process
* `iter_connections` only needs to handle one source-target pair at a time
   * after required data from one iteration is collected, rest of the data can be discarded from memory

You might wonder what would happen if we bumped up the number of source nodes. Let's see:

In [17]:
source_node_set = 'Rt_RC'
print(f'Number of source nodes: {len(edge_population.source.ids(source_node_set))}')
print(f'Number of target nodes: {len(edge_population.target.ids(target_node_set))}')

Number of source nodes: 35567
Number of target nodes: 64856


So by using `'Rt_RC'`, we roughly bumped up the number of source sodes by one order of magnitude. 

Let's see what happens when we run it with `iter_connections`: 

In [18]:
t0 = time()
it = edge_population.iter_connections(source_node_set, target_node_set, return_edge_count=True)
pairwise_syns = np.fromiter((count for _,__,count in it), dtype=int)
print_statistics(pairwise_syns)
print(f'\nRuntime: {time()-t0:.2f} seconds')

There is a total of 29455533 synapses from 'Rt_RC' to 'VPL_TC'

Synapses between source-target node pairs:
- avg: 4.70
- std: 4.04
- min: 1
- max: 102

Runtime: 100.89 seconds


As we can see, `iter_connections` was still faster than the `pathway_synapses` method was with the smaller source node set. 

Surely, the `pathway_synapses` can't be that much worse, right?  Boy, can it ever. Running the same with the `pathway_synapses` method took **over 30 minutes**. 

Obviously, we didn't include it in the notebook, but if you **truly** want to try it out yourself, feel free to do so. You have been warned.

### Lesson's learned
* If you know you'll be working with big sample sizes and big datasets, use `iter_connections`
  * if you are unsure, you can still use it, it's not _that_ much slower 
* If your code hangs seemingly forever on
  * a call to `pathway_synapses`/`pair_edges`/`afferent_edges`/`efferent_edges`, you might want to try if `iter_connections` solves your issue
  * a `pandas.DataFrame` operation, see if you can achieve the same with `iter_connections`. It might just save your day

## Conclusion
In this notebook, we learned how to and why use the iterative approach (`iter_connections`) when working with bigger datasets to avoid having our code hanging / nodes running out of memory.